In [8]:
import json, hashlib
from pathlib import Path
import numpy as np
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx, onnxruntime as ort
from onnxruntime.quantization import QuantType, quantize_dynamic

ROOT = Path('../..').resolve()
WINDOW, HORIZON = 64, 24
FEATURE_NAMES = [
    'last_close','mean_5','mean_20','std_20','momentum_3','momentum_8','ema_5','ema_10','ret_mean_5','ret_std_20'
]
EPS = 1e-6

def synthetic_series(steps=6000, start=120.0, seed=7):
    rng = np.random.default_rng(seed)
    closes = [start]
    for t in range(1, steps):
        drift = 0.04 * np.sin(t/25) + 0.004*(t/steps)
        closes.append(closes[-1] + drift + rng.normal(scale=0.6))
    return np.asarray(closes, dtype=np.float32)

def ema(arr, span):
    alpha = 2/(span+1)
    v = arr[0]
    for x in arr[1:]:
        v = alpha*x + (1-alpha)*v
    return float(v)

def featurize(window):
    w = np.asarray(window, dtype=np.float32)
    r = np.diff(w)/w[:-1]
    return np.array([
        w[-1], w[-5:].mean(), w[-20:].mean(), w[-20:].std(),
        w[-1]-w[-3], w[-1]-w[-8], ema(w[-5:],5), ema(w[-10:],10),
        r[-5:].mean(), r[-20:].std()
    ], dtype=np.float32)

def dataset(series):
    X, y, last = [], [], []
    for i in range(WINDOW, len(series)-HORIZON):
        win = series[i-WINDOW:i]
        tgt = series[i:i+HORIZON]
        last_close = win[-1]
        X.append(featurize(win))
        y.append(tgt - last_close)
        last.append(last_close)
    return np.stack(X), np.stack(y), np.asarray(last, dtype=np.float32)

series = synthetic_series()
X, y_delta, last_close = dataset(series)
split = int(len(X)*0.8)
Xtr, Xval = X[:split], X[split:]
ytr, yval = y_delta[:split], y_delta[split:]
last_val = last_close[split:]

mean, std = Xtr.mean(axis=0), Xtr.std(axis=0)
Xtr_n = (Xtr-mean)/(std+EPS)
Xval_n = (Xval-mean)/(std+EPS)

model = LinearRegression().fit(Xtr_n, ytr)
pred_val = model.predict(Xval_n) + last_val.reshape(-1,1)
true_val = yval + last_val.reshape(-1,1)
mape = np.mean(np.abs((true_val - pred_val) / np.maximum(true_val, EPS)))
mape

0.023377193

In [9]:
onnx_path = ROOT/"apps/web/public/models/forecast_minimal.onnx"
onnx_model = convert_sklearn(
    model,
    initial_types=[('input', FloatTensorType([1, Xtr.shape[1]]))],
    target_opset=17,
    final_types=[('delta', FloatTensorType([1, HORIZON]))],
)
onnx.checker.check_model(onnx_model)
onnx_path.parent.mkdir(parents=True, exist_ok=True)
onnx_path.write_bytes(onnx_model.SerializeToString())

quant_path = onnx_path.with_name('forecast_minimal.quant.onnx')
quantize_dynamic(onnx_path, quant_path, weight_type=QuantType.QInt8)

def sha256(p: Path):
    h = hashlib.sha256(); h.update(p.read_bytes()); return h.hexdigest()

manifest = {
    "modelName": "forecast_minimal",
    "modelVer": "min-0-1-0",
    "path": "/models/forecast_minimal.onnx",
    "quantPath": "/models/forecast_minimal.quant.onnx",
    "onnxSha256": sha256(onnx_path),
    "quantSha256": sha256(quant_path),
    "inputShape": [1, Xtr.shape[1]],
    "horizonSteps": HORIZON,
    "featureWindow": WINDOW,
    "tailSize": 128,
    "normalization": {"type": "zscore", "mean": mean.tolist(), "std": (std+EPS).tolist(), "epsilon": EPS},
    "features": [{"name": n} for n in FEATURE_NAMES],
    "outputs": ["delta"],
    "postprocess": "last_close_plus_delta",
    "rtol": 1e-3,
    "atol": 1e-4,
    "val_metrics": {"mape": float(mape)},
}
(ROOT/"apps/web/src/config/ml.manifest.json").write_text(json.dumps(manifest, indent=2))
manifest

{'modelName': 'forecast_minimal',
 'modelVer': 'min-0-1-0',
 'path': '/models/forecast_minimal.onnx',
 'quantPath': '/models/forecast_minimal.quant.onnx',
 'onnxSha256': '68f9832f049a376ed82c463ef03db44e68058ba426f8f81625baeacd55cf20f7',
 'quantSha256': '67f47aa3c602bb767a251ffdc617ef6569af026cf0087d51a2b4640997f34b03',
 'inputShape': [1, 10],
 'horizonSteps': 24,
 'featureWindow': 64,
 'tailSize': 128,
 'normalization': {'type': 'zscore',
  'mean': [78.76679229736328,
   78.7876205444336,
   78.86705017089844,
   1.0511113405227661,
   -0.020726416260004044,
   -0.07333444803953171,
   78.78337097167969,
   78.80610656738281,
   -9.05634296941571e-05,
   0.007382345851510763],
  'std': [9.699355125427246,
   9.70761775970459,
   9.732146263122559,
   0.4197289049625397,
   0.8503064513206482,
   1.6205174922943115,
   9.702330589294434,
   9.707200050354004,
   0.003541243262588978,
   0.0014764944789931178],
  'epsilon': 1e-06},
 'features': [{'name': 'last_close'},
  {'name': 'mean_